In [22]:
%matplotlib inline
import pandas as pd
import numpy as np

In [2]:
from sklearn.preprocessing import LabelEncoder

# Cleaning 2: processing CSS features

In this notebook we will process data from previose notebook 'Upload_prepare_data [1 step - cleaned]'. 
Here we will do the follwing:
* remove unnecessary CSS properties
* create label encoding for every categorucal feature
* remove duplicate features
* replace standard textual values of CSS properties with numeric ones
* see summary statistics for all numeric and categoriacal features

In [35]:
%store -r DATA_CL

In [36]:
data = DATA_CL

In [37]:
data.shape

(291030, 297)

Count missing values per columns

In [38]:
data.isnull().sum()

url                                       0
meta_name                                 1
text                                    942
x_coords                                  1
y_coords                                  1
block_height                              1
block_width                               2
tag                                     762
num_child                            135866
num_siblings                              2
border-top-width                          2
table-layout                              2
-webkit-shape-margin                      2
-webkit-flow-into                         2
-webkit-animation-iteration-count         2
min-height                                2
-webkit-region-fragment                   2
-webkit-highlight                         2
-webkit-column-rule-width                 2
-webkit-rtl-ordering                      2
background-size                           2
font-size                                 2
text-shadow                     

In [39]:
# del data['num_child']

Remove NA

In [40]:
data = data.dropna(subset=['block_width', 'text'])

Fill tag with no_tag value

In [41]:
data.tag = data.tag.fillna('no_tag')

/Users/jetbrains/miniconda3/lib/python3.5/site-packages/pandas/core/generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [42]:
data.shape

(290087, 297)

Remove small anount of rows which still have NA (around 10 rows)

In [43]:
data = data.dropna()

## Missing values

In [44]:
def get_na_stat(data):
    na_count_col = data.isnull().sum()
    df = na_count_col.to_frame()
    return df.sort_index()

In [45]:
na_count_col = get_na_stat(data)
na_count_col

,0
-webkit-align-content,0
-webkit-align-items,0
-webkit-align-self,0
-webkit-animation-delay,0
-webkit-animation-direction,0
-webkit-animation-duration,0
-webkit-animation-fill-mode,0
-webkit-animation-iteration-count,0
-webkit-animation-name,0
-webkit-animation-play-state,0


## Working with CSS subset

In [46]:
css = data.iloc[:,9:]

In [47]:
css.shape

(154240, 288)

### Unique values of CSS properties

Calculate number of unique values for every column

In [48]:
unique_css_val = pd.DataFrame(css.apply(lambda x: len(x.unique())), columns=['val']).reset_index().sort_values(by=['val'])

We have a lot of properties with only one uniqe values, let's remove them

In [49]:
unique_css_val[unique_css_val.val.isin([1, 3, 4])].shape

(181, 2)

In [50]:
unique_css_val.val.value_counts()

1       153
2        36
4        15
3        13
10        8
5         7
1807      5
8         5
7         3
8600      2
53        2
68        2
16        2
15        2
13        2
9         2
6         2
14        1
11        1
27        1
21        1
22        1
23        1
24        1
1519      1
46        1
149       1
1829      1
1816      1
1815      1
1810      1
1808      1
268       1
1228      1
147       1
1517      1
109       1
107       1
99        1
95        1
70        1
65        1
60        1
50        1
Name: val, dtype: int64

In [51]:
css_col_1_3_4_val = unique_css_val[unique_css_val.val.isin([1, 3, 4])].index.values

In [52]:
css[css_col_1_3_4_val].head()

,buffered-rendering,stroke-width,stroke-linejoin,-webkit-column-break-inside,fill-rule,-webkit-mask-size,-webkit-order,zoom,-webkit-column-count,-webkit-marquee-direction,...,text-rendering,background-repeat,widows,-webkit-text-stroke-width,transition-delay,border-right-style,clear,-webkit-transition-delay,orphans,border-left-style
8,auto,1.0,miter,auto,nonzero,auto,0,1,auto,auto,...,auto,repeat,auto,0px,0s,none,none,0s,auto,none
10,auto,1.0,miter,auto,nonzero,auto,0,1,auto,auto,...,auto,repeat,auto,0px,0s,none,none,0s,auto,none
11,auto,1.0,miter,auto,nonzero,auto,0,1,auto,auto,...,auto,repeat,auto,0px,0s,none,none,0s,auto,none
14,auto,1.0,miter,auto,nonzero,auto,0,1,auto,auto,...,auto,repeat,auto,0px,0s,none,none,0s,auto,none
22,auto,1.0,miter,auto,nonzero,auto,0,1,auto,auto,...,auto,repeat,auto,0px,0s,none,none,0s,auto,none


In [53]:
css = css.drop(css.columns[css_col_1_3_4_val], axis=1)

In [54]:
css.shape

(154240, 107)

In [55]:
css.head()

,num_siblings,min-height,background-size,font-size,text-shadow,color,-webkit-border-vertical-spacing,-webkit-background-composite,background-origin,list-style-type,...,text-align,fill,font-variant,border-top-right-radius,word-spacing,-webkit-text-fill-color,-webkit-font-variant-ligatures,-webkit-animation-name,-webkit-align-items,max-height
8,2,0px,auto,26px,none,"rgb(51, 51, 51)",0px,source-over,padding-box,disc,...,start,#000000,normal,0px,0px,"rgb(51, 51, 51)",normal,none,stretch,none
10,22,0px,auto,13px,none,"rgb(0, 0, 0)",0px,source-over,padding-box,disc,...,start,#000000,normal,0px,0px,"rgb(0, 0, 0)",normal,none,stretch,none
11,2,0px,auto,26px,none,"rgb(51, 51, 51)",0px,source-over,padding-box,disc,...,start,#000000,normal,0px,0px,"rgb(51, 51, 51)",normal,none,stretch,none
14,21,0px,auto,13px,none,"rgb(0, 0, 0)",0px,source-over,padding-box,disc,...,start,#000000,normal,0px,0px,"rgb(0, 0, 0)",normal,none,stretch,none
22,2,0px,auto,26px,none,"rgb(51, 51, 51)",0px,source-over,padding-box,disc,...,start,#000000,normal,0px,0px,"rgb(51, 51, 51)",normal,none,stretch,none


In [56]:
css.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 154240 entries, 8 to 1243363
Columns: 107 entries, num_siblings to max-height
dtypes: object(107)
memory usage: 127.1+ MB


In [57]:
css_obj = css[css.columns[css.dtypes == 'object']]


In [58]:
css_obj.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 154240 entries, 8 to 1243363
Columns: 107 entries, num_siblings to max-height
dtypes: object(107)
memory usage: 127.1+ MB


Remove px and rgb, we will convert it to numeric

In [59]:
css_obj = css_obj.applymap(lambda x: str(x).replace('px', ''))
css_obj = css_obj.applymap(lambda x: str(x).replace('rgb', ''))

In [60]:
css_obj = css_obj.applymap(lambda x: str(x).replace('a(', '('))

In [61]:
css[css.columns[css.dtypes == 'object']] = css_obj

In [62]:
css.shape

(154240, 107)

### 'None' values

In [63]:
count_none = lambda x: (x == 'none').sum()/len(x)

In [64]:
none_css_val = css.apply(count_none).to_frame().reset_index()

In [65]:
none_css_val.columns = ['prop', 'val']
none_css_val = none_css_val.sort_values(by='val', ascending=False)

Some CSS properties really have a lot of None values, let's remove them!

In [69]:
none_css_val.head(20)

,prop,val
104,-webkit-animation-name,0.999987
90,list-style-image,0.999793
40,-webkit-box-shadow,0.999637
41,box-shadow,0.999637
52,stroke,0.998301
58,max-width,0.997873
49,background-image,0.997076
43,border-top-style,0.996966
106,max-height,0.996480
83,border-bottom-style,0.994262


CSS properties with many None values

In [70]:
css_col_none = none_css_val.prop[none_css_val.val > 0.3].index.values

In [71]:
css_col_none

array([104,  90,  40,  41,  52,  58,  49,  43, 106,  83,   4])

In [72]:
css = css.drop(css.columns[css_col_none], axis=1)

We just reduce the number of CSS properties, number of rows didn't change

In [73]:
css.shape

(154240, 96)

In [74]:
data.shape

(154240, 297)

### Working with color columns

Working with color columns, we want to find only those columns where the last column is 'color', otherwise it's something else

All colors seems to be the same, let's consider only 'color' column

In [75]:
color_cols = [col for col in css.columns if 'color' == col[-5:]]

In [76]:
color_col = css.color

In [77]:
css_color_df = css[color_cols]

In [80]:
css_color_df.head()

,color,border-left-color,border-bottom-color,-webkit-tap-highlight-color,outline-color,border-right-color,-webkit-column-rule-color,border-top-color,-webkit-text-stroke-color,background-color,-webkit-text-emphasis-color,-webkit-text-fill-color
8,"(51, 51, 51)","(51, 51, 51)","(51, 51, 51)","(0, 0, 0, 0)","(51, 51, 51)","(51, 51, 51)","(51, 51, 51)","(51, 51, 51)","(51, 51, 51)","(0, 0, 0, 0)","(51, 51, 51)","(51, 51, 51)"
10,"(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0, 0)","(0, 0, 0)","(0, 0, 0)"
11,"(51, 51, 51)","(51, 51, 51)","(51, 51, 51)","(0, 0, 0, 0)","(51, 51, 51)","(51, 51, 51)","(51, 51, 51)","(51, 51, 51)","(51, 51, 51)","(0, 0, 0, 0)","(51, 51, 51)","(51, 51, 51)"
14,"(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0, 0)","(0, 0, 0)","(0, 0, 0)"
22,"(51, 51, 51)","(51, 51, 51)","(51, 51, 51)","(0, 0, 0, 0)","(51, 51, 51)","(51, 51, 51)","(51, 51, 51)","(51, 51, 51)","(51, 51, 51)","(0, 0, 0, 0)","(51, 51, 51)","(51, 51, 51)"


In [81]:
css_color_df.shape

(154240, 12)

List of Dataframes with expanded colors

In [82]:
def color_df_to_expand(series):
    series_exp = series.str[1:-1].str.split(',', expand=True).astype(float)
    series_exp = series_exp.iloc[:,:-1] #remove last NAN
    return series_exp

In [83]:
color_col = color_df_to_expand(color_col)

In [84]:
color_col.columns = ['color_r', 'color_g', 'color_b']

Remove all color columns except 'color' one because all colors seems to be the same

In [85]:
css = css.drop(css[color_cols], axis=1)

Append extracted colors R, G, B columns

In [86]:
css = pd.concat([color_col, css], axis=1)

In [87]:
css_filt = css[['color_r', 'color_g', 'color_b', 'font-size', 'display', 
     'font-weight', '-webkit-locale', 'width', 'height', 'font-family',
     'text-align', 'line-height']]

In [88]:
css_filt.columns = ['color_r', 'color_g', 'color_b', 'font_size', 'display', 
     'font_weight', 'locale', 'width', 'height', 'font_family',
     'text_align', 'line_height']

In [89]:
css_filt.shape

(154240, 12)

### font_size

This feature is already numeric, nothing to do with it.

In [90]:
css_filt.font_size.value_counts().head()

14    21298
13    20791
12    18929
16    18258
11    16573
Name: font_size, dtype: int64

### display

Create label encoder for 'display' feature

In [91]:
css_filt.display.value_counts()

inline                92514
block                 51704
none                   5532
inline-block           2031
table-cell             1038
table-row               674
list-item               405
table                   330
-webkit-box               6
-webkit-flex              2
table-caption             2
table-row-group           1
table-footer-group        1
Name: display, dtype: int64

In [92]:
le_css_display = LabelEncoder()
le_css_display = le_css_display.fit_transform(css_filt.display.values)
css_filt.loc[:,'display'] = le_css_display

/Users/jetbrains/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


### locale

We will filter by locale later, let's exemine the number of every locale. In future we will leave only English language

In [93]:
css_filt.locale.value_counts().head(10)

en       41945
auto     26159
de-DE    15958
en-us    14895
en-US    14758
ro        7855
de        5496
en-GB     2037
in-ID     1865
fr        1819
Name: locale, dtype: int64

Working with font_weight

### font_weight

In [96]:
css_filt.font_weight.value_counts()

400    100760
700     35382
300     11523
500      4494
600      1475
200       311
100       179
900        74
800        42
Name: font_weight, dtype: int64

In [97]:
css_filt.font_weight[css_filt.font_weight == 'bold'] = 700
css_filt.font_weight[css_filt.font_weight == 'normal'] = 400

/Users/jetbrains/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/jetbrains/miniconda3/lib/python3.5/site-packages/pandas/core/generic.py:4485: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/Users/jetbrains/miniconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_n

### font_family

In [98]:
def font_df_to_expand(series):
    series_exp = series.str.split(',', expand=True)
    series_exp = series_exp.iloc[:,0] #only family of font
    series_exp = series_exp.apply(lambda x: x.replace("'","").replace('b"',''))
    return series_exp

Working with Font family, we will left only familiy without subtypes

In [99]:
font = font_df_to_expand(css_filt.font_family)

In [100]:
le_css_font = LabelEncoder()
le_css_font = le_css_font.fit_transform(css_filt.font_family.values)
css_filt.loc[:,'font_family'] = le_css_font

/Users/jetbrains/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [101]:
css_filt.shape

(154240, 12)

In [102]:
data.shape

(154240, 297)

### text align

Working with text align, create label encoder

In [103]:
css_filt.text_align.value_counts()

start             86482
left              57652
center             8676
-webkit-left        520
right               449
justify             408
-webkit-center       49
-webkit-right         4
Name: text_align, dtype: int64

In [104]:
le_css_text_align = LabelEncoder()
le_css_text_align = le_css_text_align.fit_transform(css_filt.text_align.values)
css_filt.loc[:,'text_align'] = le_css_text_align

/Users/jetbrains/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


### width and  height

Replace value 'auto' with -1

In [105]:
css_filt[['width', 'height']] = css_filt[['width', 'height']].apply(lambda x: x.replace('auto', -1))

/Users/jetbrains/miniconda3/lib/python3.5/site-packages/pandas/core/frame.py:2378: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


### line_height

Replace value 'normal' with 1.2

In [106]:
# Source: https://developer.mozilla.org/en/docs/Web/CSS/line-height?v=control

css_filt['line_height'][css_filt['line_height'] == 'normal'] = 1.2

/Users/jetbrains/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/jetbrains/miniconda3/lib/python3.5/site-packages/pandas/core/generic.py:4485: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/Users/jetbrains/miniconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns

## Convert CSS columns to numeric

Now everything is ready for converting the sub dataframe to a numeric form.

In [107]:
num_css_col = [col for col in css_filt.columns if col != 'locale']

In [108]:
css_filt_num = css_filt[num_css_col]

In [109]:
css_filt_num = css_filt_num.convert_objects(convert_numeric=True)

/Users/jetbrains/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


In [110]:
css_filt_num.head()

,color_r,color_g,color_b,font_size,display,font_weight,width,height,font_family,text_align,line_height
8,51.0,51.0,51.0,26,3,300,-1.0,-1.0,856,7,28.0
10,0.0,0.0,0.0,13,2,400,370.0,100.0,856,7,20.0
11,51.0,51.0,51.0,26,3,300,-1.0,-1.0,856,7,28.0
14,0.0,0.0,0.0,13,3,400,-1.0,-1.0,856,7,20.0
22,51.0,51.0,51.0,26,3,300,-1.0,-1.0,856,7,28.0


Remove all CSS columns from original data

In [111]:
data = data.iloc[:,:9]

In [112]:
data.shape

(154240, 9)

In [113]:
css_filt_num.shape

(154240, 11)

In [114]:
data_css_num = pd.concat([data, css_filt_num], axis=1)

In [115]:
data_css_num.shape

(154240, 20)

In [116]:
data_css_num.head()

,url,meta_name,text,x_coords,y_coords,block_height,block_width,tag,num_child,color_r,color_g,color_b,font_size,display,font_weight,width,height,font_family,text_align,line_height
8,http://www.chicagoparkdistrict.com/parks/Galew...,name,Movie Inside the Park at Mayfair Park,15,200,60,357,span,None,51.0,51.0,51.0,26,3,300,-1.0,-1.0,856,7,28.0
10,http://www.chicagoparkdistrict.com/parks/Galew...,location,Mayfair Park\n\n \n\n 4...,15,313,100,370,p,None,0.0,0.0,0.0,13,2,400,370.0,100.0,856,7,20.0
11,http://www.chicagoparkdistrict.com/parks/Scott...,name,Night Out: Movies in the Parks at Sauganash,15,200,60,363,span,None,51.0,51.0,51.0,26,3,300,-1.0,-1.0,856,7,28.0
14,http://www.chicagoparkdistrict.com/parks/Scott...,description,The Land Before TimeFive orphan dinosaurs trav...,15,493,170,370,span,None,0.0,0.0,0.0,13,3,400,-1.0,-1.0,856,7,20.0
22,http://www.chicagoparkdistrict.com/events/movi...,name,Movies in the Park: The Smurfs,15,200,32,339,span,None,51.0,51.0,51.0,26,3,300,-1.0,-1.0,856,7,28.0


### tag

In [117]:
data_css_num.tag.value_counts().head()

span    80985
div     27368
time    18027
p        7327
h1       3760
Name: tag, dtype: int64

In [118]:
le_tag = LabelEncoder()
le_tag = le_tag.fit_transform(data_css_num.tag.values)
data_css_num.loc[:,'tag'] = le_tag

### meta_name

In [119]:
data_css_num.meta_name.value_counts()

name           63320
location       37954
startDate      33525
description    19441
Name: meta_name, dtype: int64

We will do label encoding later, no it would more convenient to see original label

In [120]:
data_css_num.iloc[:,3:]

data_num = data_css_num.iloc[:,3:].convert_objects(convert_numeric=True)

/Users/jetbrains/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  app.launch_new_instance()


In [121]:
data_css_num.iloc[:,3:] = data_num

In [122]:
DATA_NUM = data_css_num

In [123]:
DATA_NUM['locale'] = css_filt.locale

In [124]:
DATA_NUM.head()

,url,meta_name,text,x_coords,y_coords,block_height,block_width,tag,num_child,color_r,...,color_b,font_size,display,font_weight,width,height,font_family,text_align,line_height,locale
8,http://www.chicagoparkdistrict.com/parks/Galew...,name,Movie Inside the Park at Mayfair Park,15,200,60,357,25,None,51.0,...,51.0,26,3,300,-1.0,-1.0,856,7,28.0,en
10,http://www.chicagoparkdistrict.com/parks/Galew...,location,Mayfair Park\n\n \n\n 4...,15,313,100,370,21,None,0.0,...,0.0,13,2,400,370.0,100.0,856,7,20.0,en
11,http://www.chicagoparkdistrict.com/parks/Scott...,name,Night Out: Movies in the Parks at Sauganash,15,200,60,363,25,None,51.0,...,51.0,26,3,300,-1.0,-1.0,856,7,28.0,en
14,http://www.chicagoparkdistrict.com/parks/Scott...,description,The Land Before TimeFive orphan dinosaurs trav...,15,493,170,370,25,None,0.0,...,0.0,13,3,400,-1.0,-1.0,856,7,20.0,en
22,http://www.chicagoparkdistrict.com/events/movi...,name,Movies in the Park: The Smurfs,15,200,32,339,25,None,51.0,...,51.0,26,3,300,-1.0,-1.0,856,7,28.0,en


In [125]:
DATA_NUM = DATA_NUM.dropna()

In [128]:
DATA_NUM.head().iloc[:,3:]

,x_coords,y_coords,block_height,block_width,tag,num_child,color_r,color_g,color_b,font_size,display,font_weight,width,height,font_family,text_align,line_height,locale
8,15,200,60,357,25,None,51.0,51.0,51.0,26,3,300,-1.0,-1.0,856,7,28.0,en
10,15,313,100,370,21,None,0.0,0.0,0.0,13,2,400,370.0,100.0,856,7,20.0,en
11,15,200,60,363,25,None,51.0,51.0,51.0,26,3,300,-1.0,-1.0,856,7,28.0,en
14,15,493,170,370,25,None,0.0,0.0,0.0,13,3,400,-1.0,-1.0,856,7,20.0,en
22,15,200,32,339,25,None,51.0,51.0,51.0,26,3,300,-1.0,-1.0,856,7,28.0,en


In [129]:
DATA_NUM.head().iloc[:,3:].columns

Index(['x_coords', 'y_coords', 'block_height', 'block_width', 'tag',
       'num_child', 'color_r', 'color_g', 'color_b', 'font_size', 'display',
       'font_weight', 'width', 'height', 'font_family', 'text_align',
       'line_height', 'locale'],
      dtype='object')

## Save data and encoders for futher work

In [243]:
# Store clean numeric data
%store DATA_NUM

# Store tag encoding
%store le_tag

# Store css propertirs encodings
%store le_css_text_align
%store le_css_font
%store le_css_display

Stored 'DATA_NUM' (DataFrame)
Stored 'le_tag' (ndarray)
Stored 'le_css_text_align' (ndarray)
Stored 'le_css_font' (ndarray)
Stored 'le_css_display' (ndarray)


In [244]:
DATA_NUM.shape

(290074, 21)

In [245]:
import h5py
DATA_NUM.to_hdf('store.h5', 'DATA_NUM')